In [22]:
import sys
sys.path.append('/home/angsj/Repo/projects/htvs/')
sys.path.append('/home/angsj/Repo/projects/htvs/djangochem')
sys.path.append('home/angsj/Repo')
import random
import itertools

In [23]:
import os
import django

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
# The environment variables is very importatn here, make sure htvs/djangochem/ is in your path somehow
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
django.setup()

from django.contrib.auth.models import Group, Permission, User
from django.contrib.sessions.models import Session
from django.contrib.contenttypes.models import ContentType
from neuralnet.models import NetArchitecture, NeuralNetwork, NnPotential, NnPotentialStats
from jobs.models import Job, JobConfig, WorkBatch
from guardian.models import GroupObjectPermission, UserObjectPermission
from pgmols.models import AtomBasis, BasisSet, Batch, Calc, Cluster, Geom, Hessian, Jacobian, Mechanism, Method, Mol, MolGroupObjectPermission, MolSet, MolUserObjectPermission, PathImage, ProductLink, ReactantLink, Reaction, ReactionPath, ReactionType, SinglePoint, Species, Stoichiometry
from django.contrib.admin.models import LogEntry
from features.models import AtomDescriptor, BondDescriptor, ConnectivityMatrix, DistanceMatrix, Fingerprint, ProximityMatrix, SpeciesDescriptor, TrainingSet, Transformation

from django.urls import reverse
from django.conf import settings
from django.core.cache import cache
from django.utils import timezone
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery

import numpy as np
import json
import argparse
import time, subprocess
from neuralnet.utils import *

In [26]:
family = NetFamily.objects.get(name='id5945_active0')

In [19]:
# decide you training data 

geomid = []
for rp in ReactionPath.objects.filter(id=5945).distinct():
    try:
        #print('Reaction between', list(rp.reaction.values_list('reactants__smiles','products__smiles').distinct()))
        j = rp.childjobs.filter(status__in=['done',
                                            'retried']).order_by('-completetime')#.first()
             
        geomid.append(list(j.values_list("childgeoms__id", flat=True)))
     
    except:
        pass
    
#Normal Mode Sampling Geoms
    nms_geoms = Geom.objects.filter(parentjob__group__name='barriers', 
                                    parentjob__config__name='hessian_displacement_m062x_def2svp_neb_orca',
                                    parentjob__parentgeom__reactionpath=rp)
    
    print("Number of NMS Geoms: {}".format(nms_geoms.count()))
    geomid.append(list(nms_geoms.values_list("id", flat=True)))
    
merged = list(itertools.chain(*geomid))
random.shuffle(merged)
merged = merged[:130000]
merged = set(merged)
trainingset = Geom.objects.filter(id__in=merged)

print("Finishing getting {} geoms for training".format(trainingset.count()))

Number of NMS Geoms: 1600
Finishing getting 6288 geoms for training


In [28]:
train_params = {
    'batch_size': 64,
    'device': 'cuda',
    'gres': 1,
    'logger': 'csv',
    'log_every_n_epochs': 1,
    'loss_coef': '{\"energy\": 0.01, \"energy_grad\": 1.0}',
    'lr': 3e-4,
    'lr_decay': 0.5,
    'lr_min': 2e-7,
    'lr_patience': 25,
    'max_epochs': 500,
    'n_epochs': 500,
    'nprocs': 12,    
    'partitions': ['sched_mit_rafagb'],
    'split': [0.2, 0.2],
    #'trainspecies': training_smiles,
    'weightpath': '/home/angsj/models',
    'workers': 2
}

model_params = {
    'cutoff': 5.5,
    'model': 'schnet',
    'n_atom_basis': 256,
    'n_convolutions': 8,
    'n_filters': 256,
    'n_gaussians': 32,
    'trainable_gauss': True
}

architecture = NetArchitecture.objects.last()
group, isnew = Group.objects.get_or_create(name="barriers")

nn = NnPotential(
    architecture=architecture,
    repoversion='',
    group=group,
    traininghyperparams=train_params,
    networkhyperparams=model_params,
    family=family
)

nn.generation=0
nn.save()
nn.trainingset.add(*trainingset)

IntegrityError: null value in column "generation" violates not-null constraint
DETAIL:  Failing row contains (469, , 2019-11-25 21:10:38.842526+00, null, 1, 10, null, null, null, null, f, null, null, null).


<NetArchitecture: NetArchitecture object (1)>